# Data quality

In [1]:
from pathlib import Path

import pandas as pd
from ydata_profiling import ProfileReport

## Load (restricted) health data

We consider synthetic, simplified data a small number of features.

In [2]:
project_root = Path('.') / '..' / '..'
data_dir = project_root / 'data'
notebooks_dir = project_root / 'notebooks'
eda_out_dir = notebooks_dir / 'eda-output'
eda_out_dir.mkdir(exist_ok=True)


simple_customer_df = pd.read_csv(data_dir / 'simplified-customer-health.csv')
simple_customer_df.head()

,customer_id,height,weight,occupation_group_idx,gender_idx,skin_cancer,depression
0,0,162.29231,83.763466,3,0,0.0,0.0
1,1,136.25010,53.333600,2,0,0.0,0.0
2,2,162.22852,78.866710,0,0,0.0,0.0
3,3,206.18896,141.281810,2,1,0.0,0.0
4,4,168.00403,108.018810,3,1,0.0,0.0


In [3]:
aggregate_claims_df = pd.read_csv(data_dir / 'aggregate-claim.csv')
aggregate_claims_df.head()

,customer_id,agg_claim_amount,year
0,0,10.26,2022
1,1,0.00,2022
2,2,0.00,2022
3,3,0.03,2022
4,4,4.84,2022


## Explore data with [ydata-profiling]()

In [ ]:
# minimal set to True due to BinderHub resource limitations
profile_health = ProfileReport(simple_customer_df, title="Simple customer health data", minimal=True)
# Write to disk
profile_health.to_file(eda_out_dir / 'simple-health-eda.html')
# Display in notebook (note: this step may crash binderhub. A workaround is to open created html in binderhub)
# profile_health

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████| 7/7 [00:00<00:00, 118387.61it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# minimal set to True due to BinderHub resource limitations
profile_claims = ProfileReport(aggregate_claims_df, title="Aggregate claims data", minimal=True)
# Write to disk
profile_claims.to_file(eda_out_dir / 'aggregate-claims-eda.html')
# Display in notebook (note: this step may crash binderhub. A workaround is to open created html in binderhub)
# profile_claims

## Task: Data quality concerns

Given your use-case of clustering existing insurance customers into segments for pricing of private health insurance, identify data quality issues or questions for business experts in the data sets `data/simplified-customer-health.csv` and `data/aggregate-claims.csv`.

## A solution: 

Data quality issues

### data/simplified-customer-health.csv

* What is the meaning of `occupation_group_idx`? Mapping to human-readable occupation categories?
* What is the meaning of `gender_idx`? Mapping?
* Are units of `height` centimeters? Any data quality checks at point of entry?
* Same for `weight`, kilograms.
* What relationship is likely between `weight` and `height`? What do you see in the data?
* How exactly are medical condition values assigned (`skin_cancer`, `depression`)?

### data/aggregate-claims.csv

* There appear to be duplicated rows in year 2023. What is cause? What data processing action should be taking if occurs again?

## Task: Implement a basic data contract / expectation enforcement to flag duplicate records


You may either implement your code here in the notebook, or within one of project's modules (a potential solution is given in `src/workshop/possible_data_contract.py`

### Out-of-scope

Unless you have extra time now :)

* Integrate with data pipeline
* Together with business / data producer, decide what action to take, e.g. `WARNING`, `ERROR`
* Data document generation

## A solution:

Using example implementation in `src/workshop/possible_data_contract.py`

In [ ]:
from workshop.possible_data_contract import load_data_conf, get_failed_data_expectations

# (Optional) Look at project's (declarative, as data) data-exectations
data_conf = load_data_conf()
data_conf

In [ ]:
failed_expectations = get_failed_data_expectations(df=aggregate_claims_df, expectation_suite_name='ClaimData')
print(f'Number of failed expectations: {len(failed_expectations)}')


In [ ]:
for a_failed_expectation in failed_expectations:
    print(a_failed_expectation['expectation_name'])
    print(a_failed_expectation['failures'].head())